# Eksperimen Machine Learning: Wine Quality Prediction

**Author:** Syifa Fauziah  
**Course:** Membangun Sistem Machine Learning - Dicoding  
**Dataset:** UCI Wine Quality Dataset  
**Date:** December 2025

---

Notebook ini berisi tahapan lengkap eksperimen machine learning meliputi:

1. Import Libraries dan Konfigurasi
2. Data Loading dari UCI Repository
3. Exploratory Data Analysis (EDA)
4. Data Preprocessing dan Feature Engineering
5. Data Splitting (Train/Test)
6. Feature Scaling
7. Export Data dan Artifacts

---
## Dataset Introduction

### Sumber Dataset
Dataset yang digunakan dalam proyek ini adalah **UCI Wine Quality Dataset** yang tersedia di:
- **Repository:** UCI Machine Learning Repository
- **URL:** https://archive.ics.uci.edu/ml/datasets/wine+quality
- **Referensi:** P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis (2009)

### Deskripsi Dataset
Dataset ini berisi hasil analisis physicochemical dari sampel wine dari Portugal utara:
- **Red Wine:** 1,599 sampel
- **White Wine:** 4,898 sampel  
- **Total:** 6,497 sampel setelah digabungkan

### Fitur Dataset (11 Input Variables)
| No | Fitur | Deskripsi | Satuan |
|----|-------|-----------|--------|
| 1 | fixed acidity | Asam tartarat dalam wine | g/dm³ |
| 2 | volatile acidity | Asam asetat dalam wine | g/dm³ |
| 3 | citric acid | Asam sitrat untuk kesegaran | g/dm³ |
| 4 | residual sugar | Gula tersisa setelah fermentasi | g/dm³ |
| 5 | chlorides | Kandungan garam | g/dm³ |
| 6 | free sulfur dioxide | SO2 bebas untuk antimikroba | mg/dm³ |
| 7 | total sulfur dioxide | Total SO2 | mg/dm³ |
| 8 | density | Kepadatan wine | g/cm³ |
| 9 | pH | Tingkat keasaman | - |
| 10 | sulphates | Aditif antimikroba | g/dm³ |
| 11 | alcohol | Kadar alkohol | % vol |

### Target Variable
- **quality:** Skor kualitas wine berdasarkan penilaian sensori
- **Skala:** 0 (sangat buruk) hingga 10 (sangat baik)
- **Distribusi aktual:** Mayoritas skor antara 5-7

### Tujuan Proyek
Membangun model machine learning untuk memprediksi kualitas wine berdasarkan karakteristik physicochemical-nya.

---
## 1. Import Libraries dan Konfigurasi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
import json
import warnings
import os
from datetime import datetime

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.4f}'.format)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10

RANDOM_STATE = 42
TEST_SIZE = 0.2

print('Libraries imported successfully')
print(f'Pandas version: {pd.__version__}')
print(f'NumPy version: {np.__version__}')
print(f'Execution timestamp: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

---
## 2. Data Loading

Dataset diambil dari UCI Machine Learning Repository. Dataset terdiri dari dua jenis wine:
- Red Wine (1599 samples)
- White Wine (4898 samples)

Kedua dataset digabungkan dengan menambahkan kolom `wine_type` sebagai identifier.

In [ ]:
RED_WINE_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
WHITE_WINE_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

print('Fetching data from UCI Machine Learning Repository...')

try:
    df_red = pd.read_csv(RED_WINE_URL, sep=';')
    df_white = pd.read_csv(WHITE_WINE_URL, sep=';')
    print('Data fetched successfully from UCI repository')
except Exception as e:
    print(f'Error fetching data: {e}')
    raise

df_red['wine_type'] = 'red'
df_white['wine_type'] = 'white'

df = pd.concat([df_red, df_white], axis=0, ignore_index=True)

print(f'\nDataset Statistics:')
print(f'  Red wine samples: {len(df_red):,}')
print(f'  White wine samples: {len(df_white):,}')
print(f'  Total samples: {len(df):,}')
print(f'  Number of features: {len(df.columns) - 1}')

In [ ]:
output_raw_dir = '../winequality_raw'
os.makedirs(output_raw_dir, exist_ok=True)

df.to_csv(f'{output_raw_dir}/winequality_combined.csv', index=False)
df_red.drop(columns=['wine_type']).to_csv(f'{output_raw_dir}/winequality_red.csv', index=False)
df_white.drop(columns=['wine_type']).to_csv(f'{output_raw_dir}/winequality_white.csv', index=False)

print(f'Raw data saved to {output_raw_dir}/')
print('Files created:')
for f in os.listdir(output_raw_dir):
    size = os.path.getsize(f'{output_raw_dir}/{f}') / 1024
    print(f'  - {f} ({size:.1f} KB)')

In [ ]:
print('Dataset Shape:', df.shape)
print('\nColumn Names and Data Types:')
print('-' * 50)
for col in df.columns:
    print(f'  {col}: {df[col].dtype}')

In [ ]:
print('First 10 rows of the dataset:')
df.head(10)

In [ ]:
print('Dataset Info:')
df.info()

---
## 3. Exploratory Data Analysis (EDA)

Tahap EDA meliputi analisis statistik deskriptif, pengecekan missing values dan duplikat, analisis distribusi target dan fitur, serta analisis korelasi antar variabel.

### 3.1 Descriptive Statistics

In [ ]:
print('Descriptive Statistics for Numerical Features:')
desc_stats = df.describe().T
desc_stats['range'] = desc_stats['max'] - desc_stats['min']
desc_stats['cv'] = (desc_stats['std'] / desc_stats['mean']) * 100
desc_stats

In [ ]:
print('Additional Statistics:')
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

additional_stats = pd.DataFrame({
    'skewness': df[numeric_cols].skew(),
    'kurtosis': df[numeric_cols].kurtosis(),
    'median': df[numeric_cols].median(),
    'mode': df[numeric_cols].mode().iloc[0],
    'iqr': df[numeric_cols].quantile(0.75) - df[numeric_cols].quantile(0.25)
})
additional_stats

### 3.2 Missing Values Analysis

In [ ]:
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage (%)': missing_percentage
})

print('Missing Values Analysis:')
print('-' * 50)
if missing_values.sum() == 0:
    print('No missing values found in the dataset')
else:
    print(missing_df[missing_df['Missing Count'] > 0])

print(f'\nTotal missing values: {missing_values.sum()}')

### 3.3 Duplicate Analysis

In [ ]:
total_duplicates = df.duplicated().sum()
duplicate_percentage = (total_duplicates / len(df)) * 100

print('Duplicate Analysis:')
print('-' * 50)
print(f'Total duplicate rows: {total_duplicates:,}')
print(f'Percentage of duplicates: {duplicate_percentage:.2f}%')

if total_duplicates > 0:
    print('\nSample of duplicate rows:')
    display(df[df.duplicated(keep='first')].head())

### 3.4 Target Variable Analysis (Quality)

In [ ]:
print('Quality Score Distribution:')
quality_dist = df['quality'].value_counts().sort_index()
quality_pct = (quality_dist / len(df) * 100).round(2)

quality_analysis = pd.DataFrame({
    'Count': quality_dist,
    'Percentage (%)': quality_pct
})
print(quality_analysis)

In [ ]:
output_dir = 'winequality_preprocessing'
os.makedirs(output_dir, exist_ok=True)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

quality_counts = df['quality'].value_counts().sort_index()
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(quality_counts)))
bars = axes[0].bar(quality_counts.index.astype(str), quality_counts.values, color=colors, edgecolor='black', linewidth=1.2)
axes[0].set_xlabel('Quality Score')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Wine Quality Scores')
for bar, count in zip(bars, quality_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 30, 
                 str(count), ha='center', va='bottom', fontsize=9, fontweight='bold')

wine_type_counts = df['wine_type'].value_counts()
wedges, texts, autotexts = axes[1].pie(
    wine_type_counts.values, 
    labels=wine_type_counts.index.str.title(), 
    autopct='%1.1f%%',
    colors=['#8B0000', '#DAA520'],
    startangle=90,
    explode=(0.02, 0.02),
    shadow=True,
    textprops={'fontsize': 11}
)
axes[1].set_title('Distribution of Wine Types')

quality_by_type = df.groupby(['wine_type', 'quality']).size().unstack(fill_value=0)
quality_by_type.T.plot(kind='bar', ax=axes[2], color=['#8B0000', '#DAA520'], edgecolor='black', width=0.7)
axes[2].set_xlabel('Quality Score')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Quality Distribution by Wine Type')
axes[2].legend(title='Wine Type', loc='upper right')
axes[2].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.savefig(f'{output_dir}/01_quality_distribution.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print(f'Figure saved: {output_dir}/01_quality_distribution.png')

### 3.5 Feature Distributions

In [ ]:
feature_cols = [col for col in numeric_cols if col != 'quality']

n_features = len(feature_cols)
n_cols = 4
n_rows = int(np.ceil(n_features / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
axes = axes.flatten()

for idx, col in enumerate(feature_cols):
    ax = axes[idx]
    
    ax.hist(df[col], bins=40, color='steelblue', edgecolor='white', alpha=0.7, density=True)
    
    mean_val = df[col].mean()
    median_val = df[col].median()
    ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
    ax.axvline(median_val, color='green', linestyle='-.', linewidth=2, label=f'Median: {median_val:.2f}')
    
    ax.set_title(f'{col}', fontsize=11, fontweight='bold')
    ax.set_xlabel('')
    ax.set_ylabel('Density')
    ax.legend(fontsize=8, loc='upper right')

for idx in range(len(feature_cols), len(axes)):
    axes[idx].set_visible(False)

plt.suptitle('Feature Distributions with Mean and Median', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(f'{output_dir}/02_feature_distributions.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print(f'Figure saved: {output_dir}/02_feature_distributions.png')

### 3.6 Correlation Analysis

In [ ]:
numeric_df = df.select_dtypes(include=[np.number])
correlation_matrix = numeric_df.corr()

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, linewidths=0.5, square=True, cbar_kws={'shrink': 0.8},
            ax=axes[0], annot_kws={'size': 8})
axes[0].set_title('Feature Correlation Matrix (Lower Triangle)', fontsize=12, fontweight='bold')

quality_corr = correlation_matrix['quality'].drop('quality').sort_values(key=abs, ascending=True)
colors = ['#d73027' if x < 0 else '#1a9850' for x in quality_corr.values]
bars = axes[1].barh(quality_corr.index, quality_corr.values, color=colors, edgecolor='black', linewidth=0.5)
axes[1].axvline(x=0, color='black', linewidth=1)
axes[1].set_xlabel('Correlation Coefficient')
axes[1].set_title('Feature Correlations with Quality (Target)', fontsize=12, fontweight='bold')
axes[1].set_xlim(-0.6, 0.6)

for bar, val in zip(bars, quality_corr.values):
    x_pos = val + 0.02 if val >= 0 else val - 0.02
    ha = 'left' if val >= 0 else 'right'
    axes[1].text(x_pos, bar.get_y() + bar.get_height()/2, f'{val:.3f}', 
                 va='center', ha=ha, fontsize=9)

plt.tight_layout()
plt.savefig(f'{output_dir}/03_correlation_analysis.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print(f'Figure saved: {output_dir}/03_correlation_analysis.png')

In [ ]:
print('Feature Correlations with Quality (sorted by absolute value):')
print('-' * 50)
quality_correlations = correlation_matrix['quality'].drop('quality').sort_values(key=abs, ascending=False)

for feature, corr in quality_correlations.items():
    strength = 'Strong' if abs(corr) > 0.3 else 'Moderate' if abs(corr) > 0.15 else 'Weak'
    direction = 'Positive' if corr > 0 else 'Negative'
    print(f'  {feature}: {corr:.4f} ({strength} {direction})')

### 3.7 Outlier Detection with Boxplots

In [ ]:
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
axes = axes.flatten()

outlier_counts = {}

for idx, col in enumerate(feature_cols):
    ax = axes[idx]
    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    outlier_counts[col] = len(outliers)
    
    bp = ax.boxplot(df[col], vert=True, patch_artist=True,
                    boxprops=dict(facecolor='lightblue', color='navy'),
                    medianprops=dict(color='red', linewidth=2),
                    flierprops=dict(marker='o', markerfacecolor='red', markersize=4, alpha=0.5))
    
    ax.set_title(f'{col}\n(Outliers: {len(outliers)})', fontsize=10, fontweight='bold')
    ax.set_xticklabels([col])

for idx in range(len(feature_cols), len(axes)):
    axes[idx].set_visible(False)

plt.suptitle('Boxplots for Outlier Detection', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(f'{output_dir}/04_boxplots_outliers.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print(f'Figure saved: {output_dir}/04_boxplots_outliers.png')

In [ ]:
print('Outlier Summary (using IQR method):')
print('-' * 50)
outlier_df = pd.DataFrame({
    'Feature': list(outlier_counts.keys()),
    'Outlier Count': list(outlier_counts.values()),
    'Percentage (%)': [count/len(df)*100 for count in outlier_counts.values()]
}).sort_values('Outlier Count', ascending=False)

print(outlier_df.to_string(index=False))
print(f'\nTotal outlier instances: {sum(outlier_counts.values())}')

### 3.8 Feature Comparison by Wine Type

In [ ]:
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
axes = axes.flatten()

for idx, col in enumerate(feature_cols):
    ax = axes[idx]
    
    red_data = df[df['wine_type'] == 'red'][col]
    white_data = df[df['wine_type'] == 'white'][col]
    
    ax.hist(red_data, bins=30, alpha=0.6, color='#8B0000', label='Red', density=True)
    ax.hist(white_data, bins=30, alpha=0.6, color='#DAA520', label='White', density=True)
    
    ax.set_title(f'{col}', fontsize=11, fontweight='bold')
    ax.legend(fontsize=9)
    ax.set_ylabel('Density')

for idx in range(len(feature_cols), len(axes)):
    axes[idx].set_visible(False)

plt.suptitle('Feature Distributions by Wine Type', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(f'{output_dir}/05_features_by_wine_type.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print(f'Figure saved: {output_dir}/05_features_by_wine_type.png')

In [ ]:
print('Mean Feature Values by Wine Type:')
print('-' * 60)
comparison = df.groupby('wine_type')[feature_cols].mean().T
comparison['Difference'] = comparison['red'] - comparison['white']
comparison['% Difference'] = (comparison['Difference'] / comparison['white'] * 100).round(2)
comparison

### 3.9 Pairplot for Key Features

In [ ]:
key_features = ['alcohol', 'volatile acidity', 'sulphates', 'citric acid', 'quality']

g = sns.pairplot(df[key_features + ['wine_type']], hue='wine_type', 
                 palette={'red': '#8B0000', 'white': '#DAA520'},
                 diag_kind='kde', plot_kws={'alpha': 0.5, 's': 20},
                 height=2.5)
g.fig.suptitle('Pairplot of Key Features (Colored by Wine Type)', y=1.02, fontsize=14, fontweight='bold')

plt.savefig(f'{output_dir}/06_pairplot_key_features.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print(f'Figure saved: {output_dir}/06_pairplot_key_features.png')

---
## 4. Data Preprocessing

Tahap preprocessing meliputi:
1. Penghapusan data duplikat
2. Feature Engineering (pembuatan fitur baru)
3. Encoding variabel kategorikal
4. Penanganan outlier menggunakan IQR capping

### 4.1 Remove Duplicates

In [ ]:
initial_rows = len(df)
df_clean = df.drop_duplicates().reset_index(drop=True)
removed_duplicates = initial_rows - len(df_clean)

print('Duplicate Removal Results:')
print('-' * 50)
print(f'  Initial rows: {initial_rows:,}')
print(f'  Duplicates removed: {removed_duplicates:,}')
print(f'  Remaining rows: {len(df_clean):,}')
print(f'  Reduction: {(removed_duplicates/initial_rows)*100:.2f}%')

### 4.2 Feature Engineering

In [ ]:
print('Creating new features...')

def create_quality_category(quality):
    if quality <= 4:
        return 'low'
    elif quality <= 6:
        return 'medium'
    else:
        return 'high'

df_clean['quality_category'] = df_clean['quality'].apply(create_quality_category)

df_clean['total_acidity'] = df_clean['fixed acidity'] + df_clean['volatile acidity']

df_clean['bound_sulfur_dioxide'] = df_clean['total sulfur dioxide'] - df_clean['free sulfur dioxide']

df_clean['sugar_to_alcohol'] = df_clean['residual sugar'] / (df_clean['alcohol'] + 0.001)

df_clean['density_alcohol_ratio'] = df_clean['density'] / (df_clean['alcohol'] + 0.001)

df_clean['free_sulfur_ratio'] = df_clean['free sulfur dioxide'] / (df_clean['total sulfur dioxide'] + 0.001)

df_clean['acidity_to_pH'] = df_clean['total_acidity'] / (df_clean['pH'] + 0.001)

print('\nNew Features Created:')
print('-' * 60)
new_features = [
    ('quality_category', 'Categorical version of quality (low/medium/high)'),
    ('total_acidity', 'Sum of fixed and volatile acidity'),
    ('bound_sulfur_dioxide', 'Total minus free sulfur dioxide'),
    ('sugar_to_alcohol', 'Ratio of residual sugar to alcohol'),
    ('density_alcohol_ratio', 'Ratio of density to alcohol'),
    ('free_sulfur_ratio', 'Ratio of free to total sulfur dioxide'),
    ('acidity_to_pH', 'Ratio of total acidity to pH')
]

for feature, description in new_features:
    print(f'  {feature}: {description}')

In [ ]:
print('Quality Category Distribution:')
quality_cat_dist = df_clean['quality_category'].value_counts()
print(quality_cat_dist)

fig, ax = plt.subplots(figsize=(8, 5))
colors = {'low': '#d73027', 'medium': '#fee08b', 'high': '#1a9850'}
bars = ax.bar(quality_cat_dist.index, quality_cat_dist.values, 
              color=[colors[cat] for cat in quality_cat_dist.index], edgecolor='black')
ax.set_xlabel('Quality Category')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Quality Categories')

for bar, count in zip(bars, quality_cat_dist.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
            f'{count}\n({count/len(df_clean)*100:.1f}%)', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig(f'{output_dir}/07_quality_categories.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print(f'Figure saved: {output_dir}/07_quality_categories.png')

In [ ]:
print('Updated Dataset Shape:', df_clean.shape)
print('\nUpdated Columns:')
print(df_clean.columns.tolist())

In [ ]:
df_clean.head()

### 4.3 Encode Categorical Variables

In [ ]:
le_wine_type = LabelEncoder()
df_clean['wine_type_encoded'] = le_wine_type.fit_transform(df_clean['wine_type'])

le_quality_cat = LabelEncoder()
df_clean['quality_category_encoded'] = le_quality_cat.fit_transform(df_clean['quality_category'])

print('Encoding Results:')
print('-' * 50)
print('wine_type encoding:')
for cls, code in zip(le_wine_type.classes_, range(len(le_wine_type.classes_))):
    print(f'  {cls} -> {code}')

print('\nquality_category encoding:')
for cls, code in zip(le_quality_cat.classes_, range(len(le_quality_cat.classes_))):
    print(f'  {cls} -> {code}')

### 4.4 Handle Outliers using IQR Capping

In [ ]:
def cap_outliers_iqr(series, multiplier=1.5):
    """
    Cap outliers using IQR method.
    Values below Q1 - multiplier*IQR are set to lower bound.
    Values above Q3 + multiplier*IQR are set to upper bound.
    """
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    n_lower = (series < lower_bound).sum()
    n_upper = (series > upper_bound).sum()
    
    return series.clip(lower=lower_bound, upper=upper_bound), n_lower, n_upper

outlier_features = [
    'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
    'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
    'sulphates', 'total_acidity', 'bound_sulfur_dioxide', 'sugar_to_alcohol',
    'density_alcohol_ratio', 'free_sulfur_ratio', 'acidity_to_pH'
]

capping_results = []

print('Outlier Capping Results:')
print('-' * 60)

for col in outlier_features:
    if col in df_clean.columns:
        df_clean[col], n_lower, n_upper = cap_outliers_iqr(df_clean[col])
        total_capped = n_lower + n_upper
        if total_capped > 0:
            print(f'  {col}: {total_capped} values capped (lower: {n_lower}, upper: {n_upper})')
            capping_results.append({'feature': col, 'lower_capped': n_lower, 'upper_capped': n_upper})

print(f'\nTotal features processed: {len(outlier_features)}')
print(f'Features with outliers capped: {len(capping_results)}')

---
## 5. Data Splitting

Dataset dibagi menjadi training set (80%) dan test set (20%) dengan stratified sampling berdasarkan target variable (quality).

In [ ]:
feature_columns = [
    'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
    'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
    'pH', 'sulphates', 'alcohol', 'wine_type_encoded', 'total_acidity',
    'bound_sulfur_dioxide', 'sugar_to_alcohol', 'density_alcohol_ratio',
    'free_sulfur_ratio', 'acidity_to_pH'
]

X = df_clean[feature_columns].copy()
y_regression = df_clean['quality'].copy()
y_classification = df_clean['quality_category_encoded'].copy()

print('Feature and Target Shapes:')
print(f'  X (features): {X.shape}')
print(f'  y_regression (quality score): {y_regression.shape}')
print(f'  y_classification (quality category): {y_classification.shape}')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_regression, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE, 
    stratify=y_regression
)

_, _, y_train_cls, y_test_cls = train_test_split(
    X, y_classification,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y_classification
)

print('Train-Test Split Results:')
print('-' * 50)
print(f'  Training set: {X_train.shape[0]:,} samples ({(1-TEST_SIZE)*100:.0f}%)')
print(f'  Test set: {X_test.shape[0]:,} samples ({TEST_SIZE*100:.0f}%)')
print(f'  Number of features: {X_train.shape[1]}')
print(f'  Random state: {RANDOM_STATE}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

train_counts = y_train.value_counts().sort_index()
colors_train = plt.cm.Blues(np.linspace(0.4, 0.9, len(train_counts)))
bars1 = axes[0].bar(train_counts.index.astype(str), train_counts.values, color=colors_train, edgecolor='black')
axes[0].set_xlabel('Quality Score')
axes[0].set_ylabel('Frequency')
axes[0].set_title(f'Training Set Distribution (n={len(y_train):,})')
for bar, count in zip(bars1, train_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10, 
                 str(count), ha='center', fontsize=9)

test_counts = y_test.value_counts().sort_index()
colors_test = plt.cm.Oranges(np.linspace(0.4, 0.9, len(test_counts)))
bars2 = axes[1].bar(test_counts.index.astype(str), test_counts.values, color=colors_test, edgecolor='black')
axes[1].set_xlabel('Quality Score')
axes[1].set_ylabel('Frequency')
axes[1].set_title(f'Test Set Distribution (n={len(y_test):,})')
for bar, count in zip(bars2, test_counts.values):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
                 str(count), ha='center', fontsize=9)

plt.tight_layout()
plt.savefig(f'{output_dir}/08_train_test_distribution.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print(f'Figure saved: {output_dir}/08_train_test_distribution.png')

In [ ]:
print('Stratification Verification:')
print('-' * 50)

train_dist = y_train.value_counts(normalize=True).sort_index() * 100
test_dist = y_test.value_counts(normalize=True).sort_index() * 100
original_dist = y_regression.value_counts(normalize=True).sort_index() * 100

verification_df = pd.DataFrame({
    'Original (%)': original_dist,
    'Train (%)': train_dist,
    'Test (%)': test_dist
}).round(2)

print(verification_df)
print('\nStratification preserved: Distribution is consistent across splits')

---
## 6. Feature Scaling

Feature scaling dilakukan menggunakan StandardScaler untuk menormalkan distribusi fitur sehingga memiliki mean=0 dan standard deviation=1.

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_columns, index=X_test.index)

print('Feature Scaling Applied (StandardScaler)')
print('-' * 50)
print('Training set transformation: fit_transform()')
print('Test set transformation: transform()')

In [ ]:
print('Scaled Training Data Statistics:')
scaled_stats = X_train_scaled.describe().T[['mean', 'std', 'min', 'max']]
scaled_stats

In [ ]:
print('Scaler Parameters (Mean and Scale for each feature):')
print('-' * 60)
scaler_params = pd.DataFrame({
    'Feature': feature_columns,
    'Mean': scaler.mean_,
    'Scale (Std)': scaler.scale_
})
scaler_params

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sample_features = ['alcohol', 'volatile acidity', 'sulphates', 'citric acid']

for feature in sample_features:
    axes[0].hist(X_train[feature], bins=30, alpha=0.5, label=feature)
axes[0].set_xlabel('Value')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Before Scaling (Original)')
axes[0].legend()

for feature in sample_features:
    axes[1].hist(X_train_scaled[feature], bins=30, alpha=0.5, label=feature)
axes[1].set_xlabel('Standardized Value')
axes[1].set_ylabel('Frequency')
axes[1].set_title('After Scaling (StandardScaler)')
axes[1].legend()

plt.tight_layout()
plt.savefig(f'{output_dir}/09_scaling_comparison.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print(f'Figure saved: {output_dir}/09_scaling_comparison.png')

---
## 7. Save Preprocessed Data dan Artifacts

Menyimpan data hasil preprocessing dan artifacts (scaler, encoder, feature list) untuk digunakan dalam tahap modeling.

In [ ]:
print('Saving preprocessed data and artifacts...')
print('-' * 50)

X_train_scaled.to_csv(f'{output_dir}/X_train.csv', index=False)
X_test_scaled.to_csv(f'{output_dir}/X_test.csv', index=False)
print('Saved: X_train.csv, X_test.csv')

y_train.to_csv(f'{output_dir}/y_train.csv', index=False, header=['quality'])
y_test.to_csv(f'{output_dir}/y_test.csv', index=False, header=['quality'])
print('Saved: y_train.csv, y_test.csv (regression targets)')

y_train_cls.to_csv(f'{output_dir}/y_train_cls.csv', index=False, header=['quality_category'])
y_test_cls.to_csv(f'{output_dir}/y_test_cls.csv', index=False, header=['quality_category'])
print('Saved: y_train_cls.csv, y_test_cls.csv (classification targets)')

df_clean.to_csv(f'{output_dir}/winequality_preprocessed_full.csv', index=False)
print('Saved: winequality_preprocessed_full.csv')

In [ ]:
joblib.dump(scaler, f'{output_dir}/scaler.pkl')
print('Saved: scaler.pkl')

joblib.dump(le_wine_type, f'{output_dir}/label_encoder_wine_type.pkl')
print('Saved: label_encoder_wine_type.pkl')

joblib.dump(le_quality_cat, f'{output_dir}/label_encoder_quality_category.pkl')
print('Saved: label_encoder_quality_category.pkl')

joblib.dump(feature_columns, f'{output_dir}/feature_columns.pkl')
print('Saved: feature_columns.pkl')

In [ ]:
preprocessing_summary = {
    'metadata': {
        'author': 'Syifa Fauziah',
        'course': 'Membangun Sistem Machine Learning - Dicoding',
        'dataset': 'UCI Wine Quality',
        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'python_version': '3.9+'
    },
    'data_statistics': {
        'original_samples': int(len(df)),
        'red_wine_samples': int(len(df_red)),
        'white_wine_samples': int(len(df_white)),
        'duplicates_removed': int(removed_duplicates),
        'samples_after_cleaning': int(len(df_clean))
    },
    'split_info': {
        'train_samples': int(len(X_train)),
        'test_samples': int(len(X_test)),
        'train_ratio': float(1 - TEST_SIZE),
        'test_ratio': float(TEST_SIZE),
        'random_state': RANDOM_STATE,
        'stratified': True
    },
    'feature_info': {
        'n_features': len(feature_columns),
        'original_features': 11,
        'engineered_features': len(feature_columns) - 12,
        'feature_names': feature_columns
    },
    'target_info': {
        'regression_target': 'quality',
        'classification_target': 'quality_category',
        'quality_min': int(y_regression.min()),
        'quality_max': int(y_regression.max()),
        'quality_mean': float(y_regression.mean()),
        'quality_std': float(y_regression.std())
    },
    'preprocessing_steps': [
        'Load data from UCI repository',
        'Combine red and white wine datasets',
        'Remove duplicate rows',
        'Feature engineering (7 new features)',
        'Encode categorical variables',
        'Cap outliers using IQR method',
        'Train-test split (80/20, stratified)',
        'StandardScaler normalization'
    ],
    'output_files': {
        'training_features': 'X_train.csv',
        'test_features': 'X_test.csv',
        'training_target_regression': 'y_train.csv',
        'test_target_regression': 'y_test.csv',
        'training_target_classification': 'y_train_cls.csv',
        'test_target_classification': 'y_test_cls.csv',
        'full_preprocessed': 'winequality_preprocessed_full.csv',
        'scaler': 'scaler.pkl',
        'wine_type_encoder': 'label_encoder_wine_type.pkl',
        'quality_category_encoder': 'label_encoder_quality_category.pkl',
        'feature_list': 'feature_columns.pkl'
    }
}

with open(f'{output_dir}/preprocessing_summary.json', 'w') as f:
    json.dump(preprocessing_summary, f, indent=2)

print('\nSaved: preprocessing_summary.json')

In [ ]:
print('\nOutput Directory Contents:')
print('-' * 50)
total_size = 0
for f in sorted(os.listdir(output_dir)):
    filepath = f'{output_dir}/{f}'
    size = os.path.getsize(filepath)
    total_size += size
    print(f'  {f}: {size/1024:.1f} KB')

print(f'\nTotal size: {total_size/1024:.1f} KB ({total_size/1024/1024:.2f} MB)')

---
## 8. Summary dan Kesimpulan

In [ ]:
print('=' * 70)
print('PREPROCESSING PIPELINE SUMMARY')
print('=' * 70)

print('\n1. DATA LOADING')
print(f'   - Source: UCI Machine Learning Repository')
print(f'   - Red wine samples: {len(df_red):,}')
print(f'   - White wine samples: {len(df_white):,}')
print(f'   - Total samples: {len(df):,}')

print('\n2. DATA CLEANING')
print(f'   - Duplicates removed: {removed_duplicates:,}')
print(f'   - Final samples: {len(df_clean):,}')

print('\n3. FEATURE ENGINEERING')
print(f'   - Original features: 11')
print(f'   - New features created: {len(feature_columns) - 12}')
print(f'   - Total features: {len(feature_columns)}')

print('\n4. DATA SPLITTING')
print(f'   - Training set: {len(X_train):,} samples ({(1-TEST_SIZE)*100:.0f}%)')
print(f'   - Test set: {len(X_test):,} samples ({TEST_SIZE*100:.0f}%)')
print(f'   - Stratified by: quality score')

print('\n5. FEATURE SCALING')
print(f'   - Method: StandardScaler')
print(f'   - Training set: fit_transform()')
print(f'   - Test set: transform()')

print('\n6. OUTPUT FILES')
print(f'   - Location: {output_dir}/')
print(f'   - Data files: 7 CSV files')
print(f'   - Model artifacts: 4 PKL files')
print(f'   - Visualizations: 9 PNG files')
print(f'   - Summary: 1 JSON file')

print('\n' + '=' * 70)
print('PREPROCESSING COMPLETED SUCCESSFULLY')
print('Data is ready for model training')
print('=' * 70)

In [ ]:
verification_checks = {
    'No missing values in X_train': X_train_scaled.isnull().sum().sum() == 0,
    'No missing values in X_test': X_test_scaled.isnull().sum().sum() == 0,
    'Train-test dimensions match': X_train_scaled.shape[1] == X_test_scaled.shape[1],
    'y_train matches X_train length': len(y_train) == len(X_train_scaled),
    'y_test matches X_test length': len(y_test) == len(X_test_scaled),
    'Feature scaling applied': abs(X_train_scaled.mean().mean()) < 0.01,
    'All artifacts saved': all(os.path.exists(f'{output_dir}/{f}') 
                               for f in ['scaler.pkl', 'X_train.csv', 'y_train.csv'])
}

print('\nFinal Verification Checks:')
print('-' * 50)
all_passed = True
for check, passed in verification_checks.items():
    status = 'PASS' if passed else 'FAIL'
    symbol = '[OK]' if passed else '[X]'
    print(f'  {symbol} {check}: {status}')
    if not passed:
        all_passed = False

print('\n' + '-' * 50)
if all_passed:
    print('All verification checks PASSED')
else:
    print('Some verification checks FAILED - please review')